# Link prediction problem

- To build a model for predicting new protein-protein interaction

- Reference : [Stanford SNAP tutorial](http://snap.stanford.edu/deepnetbio-ismb/ipynb/Graph+Convolutional+Prediction+of+Protein+Interactions+in+Yeast.html)

- Tensflow 2.0 version

In [1]:
from __future__ import division
from __future__ import print_function

import time

import numpy as np
import scipy.sparse as sp

import networkx as nx
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from numba import cuda

import warnings
warnings.filterwarnings('ignore')

In [2]:
seed = 123
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
LEARNING_RATE = 0.01
EPOCHS = 20
HIDDEN1 = 32
HIDDEN2 = 16
DROPOUT = 0.1

## Various utility functions

In [4]:
def load_data():
    g = nx.read_edgelist('yeast.edgelist')
    adj = nx.adjacency_matrix(g)
    return adj

def sparse_dropout(x, keep_prob, noise_shape):
    noise_shape = [noise_shape]
    random_tensor = keep_prob
    random_tensor += tf.random.uniform(noise_shape)
    dropout_mask = tf.cast(tf.floor(random_tensor),dtype=tf.bool)
    pre_out = tf.sparse.retain(x, dropout_mask)
    return pre_out * (1./keep_prob)

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape
    
def normalize_adj(adj):
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

def preprocess_adj(adj):
    adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
    return sparse_to_tuple(adj_normalized)

def dot(x, y, sparse=False):
    if sparse:
        res = tf.sparse.sparse_dense_matmul(x, y)
    else:
        res = tf.matmul(x, y)
    return res

In [5]:
def mask_test_edges(adj):
    # Function to build test set with 2% positive links
    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()

    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    print('number of triangle upper mtx:', len(edges))
    print('number of edges_all:', len(edges_all))
    num_test = int(np.floor(edges.shape[0] / 50.))
    num_val = int(np.floor(edges.shape[0] / 50.))

    all_edge_idx = list(range(edges.shape[0]))
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)
    print('num_train_edges:', len(train_edges))
    print('num_val_edges:', len(val_edges))
    print('num_test_edges:', len(test_edges))

    def ismember(a, b):
        rows_close = np.all((a - b[:, None]) == 0, axis=-1)
        return np.any(rows_close)

    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        n_rnd = len(test_edges) - len(test_edges_false)
        rnd = np.random.randint(0, adj.shape[0], size=2 * n_rnd)
        idxs_i = rnd[:n_rnd]                                        
        idxs_j = rnd[n_rnd:]
        for i in range(n_rnd):
            idx_i = idxs_i[i]
            idx_j = idxs_j[i]
            if idx_i == idx_j:
                continue
            if ismember([idx_i, idx_j], edges_all):
                continue
            if test_edges_false:
                if ismember([idx_j, idx_i], np.array(test_edges_false)):
                    continue
                if ismember([idx_i, idx_j], np.array(test_edges_false)):
                    continue
            test_edges_false.append([idx_i, idx_j])

    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        n_rnd = len(val_edges) - len(val_edges_false)
        rnd = np.random.randint(0, adj.shape[0], size=2 * n_rnd)
        idxs_i = rnd[:n_rnd]                                        
        idxs_j = rnd[n_rnd:]
        for i in range(n_rnd):
            idx_i = idxs_i[i]
            idx_j = idxs_j[i]
            if idx_i == idx_j:
                continue
            if ismember([idx_i, idx_j], train_edges):
                continue
            if ismember([idx_j, idx_i], train_edges):
                continue
            if ismember([idx_i, idx_j], val_edges):
                continue
            if ismember([idx_j, idx_i], val_edges):
                continue
            if val_edges_false:
                if ismember([idx_j, idx_i], np.array(val_edges_false)):
                    continue
                if ismember([idx_i, idx_j], np.array(val_edges_false)):
                    continue
            val_edges_false.append([idx_i, idx_j])

    # Re-build adj matrix
    data = np.ones(train_edges.shape[0])
    adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    return adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false

## Data loading

### Adj matrix

In [6]:
adj = load_data()
num_nodes = adj.shape[0]
num_edges = adj.sum()

print(num_nodes)
print(num_edges)

6526
1062675


In [7]:
# Store original adjacency matrix (without diagonal entries) for later
adj_orig = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
adj_orig.eliminate_zeros()

adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)
adj = adj_train

number of triangle upper mtx: 530495
number of edges_all: 1060990
num_train_edges: 509277
num_val_edges: 10609
num_test_edges: 10609


In [8]:
print('train_edges:', len(train_edges))
print('val_edges:', len(val_edges))
print('val_edges_false:', len(val_edges_false))
print('test_edges:', len(test_edges))
print('test_edges_false:', len(test_edges_false))

train_edges: 509277
val_edges: 10609
val_edges_false: 10609
test_edges: 10609
test_edges_false: 10609


In [9]:
adj_norm = preprocess_adj(adj)
adj_norm

(array([[   0,    0],
        [   1,    0],
        [   2,    0],
        ...,
        [4582, 6525],
        [5312, 6525],
        [6525, 6525]], dtype=int32),
 array([0.00104384, 0.00239487, 0.00150968, ..., 0.03594426, 0.04454354,
        0.05555556]),
 (6526, 6526))

### Feature matrix (Featureless)

In [10]:
features = sparse_to_tuple(sp.identity(num_nodes))
num_features = features[2][1]
features_nonzero = features[1].shape

print(num_features)
print(features_nonzero)

6526
(6526,)


## Model

In [11]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow import keras

In [74]:
class GraphConvolution(layers.Layer):
    def __init__(self, input_dim, units, num_features_nonzero, issparse=False, dropout=0., act=tf.nn.relu, **kwargs):
        super(GraphConvolution, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.units = units
        self.num_features_nonzero = num_features_nonzero
        self.dropout = dropout
        self.act = act
        self.issparse = issparse
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(self.input_dim, self.units),
                                 initializer='random_normal',
                                 trainable=True)
        print('Shape of weights:', self.w.shape)

    def call(self, inputs, training=None):
        x, adj = inputs
        
        if training is not False and self.issparse:
            x = sparse_dropout(x, 1-self.dropout, self.num_features_nonzero)
        elif training is not False:
            x = tf.nn.dropout(x, self.dropout)
            
        x = dot(x, self.w, sparse=self.issparse)
        x = dot(adj, x, sparse=True)  # adj는 tf.SparseTensor
        outputs = self.act(x)
        
        return outputs
    
class InnerProductDecoder:
    def __init__(self, input_dim, dropout=0., act=tf.nn.sigmoid):
        self.issparse = False
        self.dropout = dropout
        self.act = act
        
    # 왜 x^Tx로 했을까?
    def __call__(self, inputs, training=None):
        x, _ = inputs
        
        x = tf.nn.dropout(x, self.dropout)
        x_t = tf.transpose(x)
        x = tf.matmul(x, x_t)
        x = tf.reshape(x, [-1])
        outputs = self.act(x)
                
        return outputs

In [75]:
class GCN(keras.Model):
    
    def __init__(self, input_dim, num_features_nonzero, **kwargs):
        super(GCN, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.num_features_nonzero = num_features_nonzero
        self.layers_ = []
        
        print('input dim:', input_dim)
        print('num_features_nonzero:', num_features_nonzero)
        
        self._build()
        
        for p in self.trainable_variables:
            print(p.name, p.shape)
            
    def _build(self):
        self.layers_.append(GraphConvolution(input_dim=self.input_dim,
                                             units=HIDDEN1,
                                             num_features_nonzero=self.num_features_nonzero,
                                             issparse=True,
                                             dropout=0.1))
        
        self.layers_.append(GraphConvolution(input_dim=HIDDEN1,
                                             units=HIDDEN2,
                                             num_features_nonzero=self.num_features_nonzero,
                                             issparse=False,
                                             dropout=0.1,
                                             act=lambda x: x))
        
        self.layers_.append(InnerProductDecoder(input_dim=HIDDEN1,
                                                dropout=0.1,
                                                act=lambda x: x))
    
    def call(self, inputs, training=None):
        x, labels, adj, num_nodes, num_edges = inputs
        
        outputs = [x]
        for layer in self.layers_:
            hidden = layer((outputs[-1], adj), training)
            outputs.append(hidden)

        output = outputs[-1]

        # Weight decay loss
        loss = tf.zeros([])
        for var in self.layers_[0].trainable_variables:
            loss += 5e-4 * tf.nn.l2_loss(var)

        # Cost
        pos_weight = tf.cast((num_nodes**2 - num_edges) / num_edges, dtype=tf.float32)
        norm = tf.cast(num_nodes**2 / (num_nodes**2 - num_edges) * 2, dtype=tf.float32)
        loss = norm * tf.reduce_mean(
            tf.nn.weighted_cross_entropy_with_logits(logits=output, labels=labels, pos_weight=pos_weight))
        
        return loss, outputs

# Training

### 모델

In [82]:
features = sparse_to_tuple(sp.identity(num_nodes))
num_features = features[2][1]
features_nonzero = features[1].shape[0]

print(num_features)
print(features_nonzero)

model = GCN(features[2][1], features[1].shape[0])
features = tf.SparseTensor(*features)

6526
6526
input dim: 6526
num_features_nonzero: 6526


In [15]:
adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = sparse_to_tuple(adj_label)

### Tensor

In [16]:
# print(tf.cast(tf.SparseTensor(*adj_norm), dtype=tf.float32))
adj_norm = tf.SparseTensor(*adj_norm)

In [17]:
adj_label = tf.SparseTensor(*adj_label)
adj_label = tf.reshape(tf.sparse.to_dense(adj_label, validate_indices=False), [-1])

In [19]:
optimizer = optimizers.Adam(lr=1e-2)

In [79]:
epochs = 20

In [80]:
def get_roc_score(edges_pos, edges_neg):
    _, outputs = model((features, adj_label, adj_norm, num_nodes, num_edges), training=False)
    
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    _, outputs = model((features, adj_label, adj_norm, num_nodes, num_edges), training=False)
    emb = outputs[2].numpy()
    adj_rec = np.dot(emb, emb.T)
        
    preds = []
    pos = []
    for e in edges_pos:
        preds.append(sigmoid(adj_rec[e[0], e[1]]))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds))])
    
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)
    
    return roc_score, ap_score

In [83]:
for epoch in range(epochs):
    curr_time = time.time()
    with tf.GradientTape() as t:
        loss, _ = model((features, adj_label, adj_norm, num_nodes, num_edges))
    grads = t.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    roc_score, ap_score = get_roc_score(val_edges, val_edges_false)
    
    print("Epoch:", '%04d' % (epoch + 1), 
          "train_loss=", "{:.5f}".format(loss),
          "val_roc=", "{:.5f}".format(roc_score),
          "val_ap=", "{:.5f}".format(ap_score),
          "time=", "{:.5f}".format(time.time() - curr_time))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Shape of weights: (6526, 32)
Shape of weights: (32, 16)
Epoch: 0001 train_loss= 2.72480 val_roc= 0.76218 val_ap= 0.68778 time= 1.11618
Epoch: 0002 train_loss= 2.72479 val_roc= 0.77556 val_ap= 0.71130 time= 1.10709
Epoch: 0003 train_loss= 2.72478 val_roc= 0.80676 val_ap= 0.76764 time= 1.12801
Epoch: 0004 train_loss= 2.72466 val_roc= 0.85559 val_ap= 0.84723 time= 1.07825
Epoch: 0005 train_loss= 2.72313 val_roc= 0.87665 val_ap= 0.86812 time= 1.09746
Epoch: 0006 train_loss= 2.69961 val_roc= 0.87845 val_ap= 

In [84]:
get_roc_score(test_edges, test_edges_false)

(0.8736581635464941, 0.8599647852129776)